In [ ]:
%cd ../..

In [2]:
import torch

torch.manual_seed(0)

if torch.cuda.is_available():
    device = 'cuda'
if torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

## Statistical metrics

In [ ]:
import torch
from torch import nn

class MeanDiffRatio(nn.Module):
    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor):
        return (y_pred.mean() - y_true.mean()) / y_true.mean()
    
metric = MeanDiffRatio()
metric(torch.tensor([1, 1.5, 3.], device=device), torch.tensor([1., 2., 3.], device=device))

In [ ]:
import torch
from torch import nn

class StdDiffRatio(nn.Module):
    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor):
        return (y_pred.std() - y_true.std()) / y_true.std()
    
metric = StdDiffRatio(device=device)
metric(torch.tensor([2., 2., 2.], device=device), torch.tensor([0., 2., 3.], device=device))

## Example output metrics

In [61]:
import os
import torch
from torch import nn
from src.tool import visual

class PedarHeatmap(nn.Module):
    def __init__(
            self,
            opt,
            mask_path: str = 'data/processed/left_foot_mask.png',
            sense_range: float = 600,
            export_folder: str = 'sample',
            ):
        super().__init__()
        self.opt = opt
        self.mask_path = mask_path
        self.sense_range = sense_range
        self.export_folder = export_folder

        # check if the export directory exists
        # if not, create it
        export_dir = f'{self.opt.path}/{self.export_folder}'
        
        if not os.path.exists(export_dir):
            os.makedirs(export_dir)

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor, dataset_name: str, batch: int):
        # export prediction samples
        for idx, y in enumerate(y_pred.reshape(-1, 198)):
            visual.draw_heatmap(
                arr=y.cpu().detach().numpy() * self.sense_range,
                l_mask_path=self.mask_path,
                is_show=False,
                is_export=True,
                export_path=f'{self.opt.path}/{self.export_folder}/{dataset_name}-{batch}-{idx}-pred.png',
                )
        
        # export groud-truth samples
        for idx, y in enumerate(y_true.reshape(-1, 198)):
            visual.draw_heatmap(
                arr=y.cpu().detach().numpy() * self.sense_range,
                l_mask_path=self.mask_path,
                is_show=False,
                is_export=True,
                export_path=f'{self.opt.path}/{self.export_folder}/{dataset_name}-{batch}-{idx}-true.png',
                )

In [62]:
from src.tool import config

opt = config.load_config('experiment/mlp-0924')
opt.path = 'notebook/prototype/output'
metric = PedarHeatmap(opt)

In [ ]:
y_pred, y_true = torch.randn(2, 5, 198), torch.randn(2, 5, 198)
metric(y_pred, y_true, 'train', 0)